In [ ]:
import numpy as np
from scipy.integrate import complex_ode
import matplotlib.pyplot as plt

# Parameters
delta_system = 1.0  # Driving field frequency
t1 = 10  # Final time for integration
dt = 0.01  # Time step for integration

# Define ranges for delta and lambda
omega_vals = np.linspace(-10, 10, 100)  # Detuning range
lambda_vals = np.linspace(0, 10, 100)  # Coupling strength range

# Create a meshgrid for delta and lambda
Omega, Lambda = np.meshgrid(omega_vals, lambda_vals)
P1 = np.zeros_like(Delta)  # Initialize the array for P1

# Loop over all combinations of delta and lambda
for i in range(Delta.shape[0]):
    for j in range(Delta.shape[1]):
        # Parameters for the current (delta, lambda) combination
        lambda_val = Lambda[i, j]  # Coupling constant
        omega = Delta[i, j]

        initial_value = [1.0, 0.0]  # Initial values for c0(0) = 1, c1(0) = 0

        # Define the system of coupled differential equations
        def system(t, y):
            c0, c1 = y[0], y[1]
            # Define the equations for dc0/dt and dc1/dt
            dc0_dt = 1j * lambda_val * (np.exp(1j * (omega - delta_system) * t) + np.exp(-1j * (omega + delta_system) * t)) * c1 / 2.0
            dc1_dt = 1j * lambda_val * (np.exp(1j * (omega + delta_system) * t) + np.exp(-1j * (omega - delta_system) * t)) * c0 / 2.0
            return [dc0_dt, dc1_dt]

        # Solve the system of ODEs using the DOP853 solver
        r = complex_ode(system)
        r.set_initial_value(initial_value, 0)  # Set initial conditions

        # Integrate the system over time
        while r.successful() and r.t < t1:
            r.integrate(r.t + dt)

        # Extract the final probability for the excited state
        c1 = r.y[1]
        P1[i, j] = np.abs(c1)**2  # Store the probability

        # Optionally, you can print the values for debugging
        # print(f"Final values at delta={delta}, lambda={lambda_val}")
        # print(f"c0 = {r.y[0]}, c1 = {r.y[1]}")
        # print(f"P1 = {P1[i, j]}")

# Plot the results
plt.figure(figsize=(10, 6))
contour = plt.contourf(Delta, Lambda, P1, levels=100, cmap='inferno')
plt.colorbar(contour, label=r'$P_1$')
plt.xlabel(r"$\omega$")
plt.ylabel(r"$\lambda$")
plt.title(r"Probability $P_1$ at $t=10$ s")
plt.show()